In [32]:
import pandas as pd
import numpy as np
import warnings
import pickle as pkl
from tqdm import tqdm
from collections import Counter

warnings.filterwarnings('ignore')

In [33]:
is_save = False

In [34]:
course = pd.read_csv('./knowledgeGraph/entity/course.csv', sep=',')

In [35]:
course_id_set = set(course['course_id'])

In [43]:
307 in course_id_set

True

user action

In [4]:
user_action_df = pd.read_csv('./interaction/challenge_interaction.csv', sep=',')
user_action_df = user_action_df[['user_id', 'challenge_id', 'open_time']]
len(user_action_df)

2532524

In [6]:
min(user_action_df['open_time']), max(user_action_df['open_time'])

('2018-01-01 00:24:44', '2019-12-30 23:59:53')

In [4]:
exe_chge = pd.read_csv('./knowledgeGraph/relation/exercise_challenge.csv', sep=',')

In [5]:
user_action = pd.merge(user_action_df, exe_chge, on=['challenge_id'])
len(user_action)

2532524

In [59]:
user_action_df = user_action.drop_duplicates(['user_id', 'exercise_id'], keep='first')
len(user_action_df)

647665

In [60]:
user_counter = Counter(user_action_df['user_id'])
selected_user_set = set(dict(filter(lambda x: x[1] >= 4, user_counter.items())).keys())

exe_counter = Counter(user_action_df['exercise_id'])
selected_exe_set = set(dict(filter(lambda x: x[1] >= 4, exe_counter.items())).keys())

len(selected_user_set), len(selected_exe_set)

(44523, 1127)

In [61]:
selected_user_action_df = user_action_df[user_action_df['exercise_id'].isin(selected_exe_set) &
                                         user_action_df['user_id'].isin(selected_user_set)]
len(selected_user_action_df)

641744

In [62]:
selected_user_set = set(selected_user_action_df['user_id'])
selected_exe_set = set(selected_user_action_df['exercise_id'])
selected_chge_set = set(selected_user_action_df['challenge_id'])

In [63]:
len(selected_user_action_df) / len(selected_user_set)

14.413763672708487

In [64]:
selected_exe_chge = exe_chge[exe_chge['exercise_id'].isin(selected_exe_set) & \
                             exe_chge['challenge_id'].isin(selected_chge_set)]

In [65]:
# discipline and sub_discipline

dpl_exe = pd.read_csv('./knowledgeGraph/relation/discipline_exercise.csv', sep=',')
dpl_exe.columns = ['exercise_id', 'sub_discipline_id', 'create_at']
selected_sub_dpl_exe = dpl_exe[dpl_exe['exercise_id'].isin(selected_exe_set)]
selected_sub_dpl_set = set(selected_sub_dpl_exe['sub_discipline_id'])

sub_dpl = pd.read_csv('./knowledgeGraph/relation/sub_discipline.csv', sep=',')
selected_sub_dpl_dpl = sub_dpl[sub_dpl['sub_discipline_id'].isin(selected_sub_dpl_set)]
selected_dpl_set = set(selected_sub_dpl_dpl['discipline_id'])

In [66]:
# exercise-teacher
exe_teacher = pd.read_csv('./knowledgeGraph/relation/exercise_teacher.csv', sep=',')
selected_exe_teacher = exe_teacher[exe_teacher['exercise_id'].isin(selected_exe_set)]
selected_exe_teacher.columns = ['exercise_id', 'teacher_id']

selected_teacher_set = set(selected_exe_teacher['teacher_id'])

In [67]:
# student-institution
stu_ins = pd.read_csv('./knowledgeGraph/relation/student_institution.csv', sep=',')

selected_stu_ins = stu_ins[stu_ins['student_id'].isin(selected_user_set)]
selected_stu_ins[['school_id']] = selected_stu_ins[['school_id']].fillna(max(selected_stu_ins['school_id']) + 1)
selected_stu_ins[['school_id']] = selected_stu_ins[['school_id']].astype(int)

In [68]:
# teacher-institution
teacher_ins = pd.read_csv('./knowledgeGraph/relation/teacher_institution.csv', sep=',')
selected_teacher_ins = teacher_ins[teacher_ins['teacher_id'].isin(selected_teacher_set)]
selected_teacher_ins[['school_id']] = selected_teacher_ins[['school_id']].fillna(max(selected_teacher_ins['school_id']) + 1)
selected_teacher_ins[['school_id']] = selected_teacher_ins[['school_id']].astype(int)

In [69]:
selected_school_set = set(set(selected_stu_ins['school_id']) | set(selected_teacher_ins['school_id']))

In [70]:
# # challenge-teacher
# chge_teacher = pd.read_csv('./knowledgeGraph/relation/challenge_teacher.csv', sep=',')
# chge_teacher.columns = ['challenge_id', 'teacher_id']
# selected_chge_teacher = chge_teacher[chge_teacher['challenge_id'].isin(selected_chge_set) & \
#                                      chge_teacher['teacher_id'].isin(selected_teacher_set)]

In [71]:
# challenge-topic
chge_topic = pd.read_csv('./knowledgeGraph/relation/challenge_topic.csv', sep=',')
selected_chge_topic = chge_topic[chge_topic['challenge_id'].isin(selected_chge_set)]

selected_topic_set = set(selected_chge_topic['topic_id'])

In [72]:
exe_topic = pd.merge(selected_exe_chge, selected_chge_topic, on='challenge_id')

In [73]:
selected_exe_topic = exe_topic[['exercise_id', 'topic_id']].drop_duplicates(keep='first')

In [74]:
print('n_user:', len(selected_user_set))
print('n_exercise:', len(selected_exe_set))
print('n_discipline:', len(selected_dpl_set))
print('n_sub_discipline:', len(selected_sub_dpl_set))
print('n_challenge:', len(selected_chge_set))
print('n_topic:', len(selected_topic_set))
print('n_teacher:', len(selected_teacher_set))
print('n_school:', len(selected_school_set))

print('n_user_challenge:', len(selected_user_action_df))
print('n_user_exercise:', len(selected_user_action_df))
# print('n_challenge_teacher:', len(selected_chge_teacher))
print('n_challenge_topic:', len(selected_chge_topic))
print('n_exercise_sub_discipline:', len(selected_sub_dpl_exe))
print('n_exercise_challenge:', len(selected_exe_chge))
print('n_exercise_teacher:', len(selected_exe_teacher))
print('n_student_school:', len(selected_stu_ins))
print('n_discipline_sub_discipline:', len(selected_sub_dpl_dpl))
print('n_teacher_school:', len(selected_teacher_ins))

n_user: 44523
n_exercise: 1127
n_discipline: 13
n_sub_discipline: 58
n_challenge: 1251
n_topic: 1227
n_teacher: 142
n_school: 1168
n_user_challenge: 641744
n_user_exercise: 641744
n_challenge_topic: 2765
n_exercise_sub_discipline: 2749
n_exercise_challenge: 1251
n_exercise_teacher: 1127
n_student_school: 44524
n_discipline_sub_discipline: 58
n_teacher_school: 142


encoding

In [75]:
def data2txt(data, path):
    with open(path, 'w') as f:
        for line in data:
            line_str = ' '.join(str(x) for x in line) + '\n'
            f.write(line_str)

In [76]:
user_id_map = [['org_user_id', 'new_user_id']]
user_id_dict = {}
id = 0
for u in selected_user_set:
    user_id_map.append([u, id])
    user_id_dict[u] = id
    id += 1

if is_save:
    data2txt(user_id_map, '../user_list.txt')

In [77]:
video_id_map = [['org_video_id', 'new_video_id']]
entity_id_dict = {}
video_id_dict = {}

id = 0
for v in selected_exe_set:
    video_id_map.append([v, id])
    entity_id_dict[v] = id
    video_id_dict[v] = id
    id += 1

if is_save:
    data2txt(video_id_map, '../video_list.txt')

entity_id_map = [['org_entity_id', 'new_entity_id']]
entity_id_map += video_id_map[1:]

dpl_id_dict = {}
for dpl in selected_dpl_set:
    entity_id_map.append([dpl, id])
    entity_id_dict[dpl] = id
    dpl_id_dict[dpl] = id
    id += 1

sub_dpl_id_dict = {}
for sub_dpl in selected_sub_dpl_set:
    entity_id_map.append([sub_dpl, id])
    entity_id_dict[sub_dpl] = id
    sub_dpl_id_dict[sub_dpl] = id
    id += 1

chge_id_dict = {}
for chge in selected_chge_set:
    entity_id_map.append([chge, id])
    entity_id_dict[chge] = id
    chge_id_dict[chge] = id
    id += 1

topic_id_dict = {}
for topic in selected_topic_set:
    entity_id_map.append([topic, id])
    entity_id_dict[topic] = id
    topic_id_dict[topic] = id
    id += 1

teacher_id_dict = {}
for teacher in selected_teacher_set:
    entity_id_map.append([teacher, id])
    entity_id_dict[teacher] = id
    teacher_id_dict[teacher] = id
    id += 1

school_id_dict = {}
for school in selected_school_set:
    entity_id_map.append([school, id])
    entity_id_dict[school] = id
    school_id_dict[school] = id
    id += 1

if is_save:
    data2txt(entity_id_map, '../entity_list.txt')

knowledge graph

In [78]:
kg = []
r_id = 0
relation_list = []

# selected_chge_teacher['new_challenge_id'] = selected_chge_teacher['challenge_id'].map(chge_id_dict)
# selected_chge_teacher['new_teacher_id'] = selected_chge_teacher['teacher_id'].map(teacher_id_dict)
# selected_chge_teacher['relation'] = r_id
# kg = selected_chge_teacher[['new_challenge_id', 'relation', 'new_teacher_id']].values
# relation_list.append(['challenge-teacher', r_id])

# r_id += 1
selected_exe_topic['new_exercise_id'] = selected_exe_topic['exercise_id'].map(video_id_dict)
selected_exe_topic['new_topic_id'] = selected_exe_topic['topic_id'].map(topic_id_dict)
selected_exe_topic['relation'] = r_id
kg = selected_exe_topic[['new_exercise_id', 'relation', 'new_topic_id']].values
relation_list.append(['exercise-topic', r_id])

r_id += 1
selected_sub_dpl_exe['new_sub_discipline_id'] = selected_sub_dpl_exe['sub_discipline_id'].map(sub_dpl_id_dict)
selected_sub_dpl_exe['new_exercise_id'] = selected_sub_dpl_exe['exercise_id'].map(video_id_dict)
selected_sub_dpl_exe['relation'] = r_id
kg = np.concatenate([kg, selected_sub_dpl_exe[['new_exercise_id', 'relation', 'new_sub_discipline_id']].values])
relation_list.append(['sub_discipline-exercise', r_id])

# r_id += 1
# selected_exe_chge['new_exercise_id'] = selected_exe_chge['exercise_id'].map(video_id_dict)
# selected_exe_chge['new_challenge_id'] = selected_exe_chge['challenge_id'].map(chge_id_dict)
# selected_exe_chge['relation'] = r_id
# kg = np.concatenate([kg, selected_exe_chge[['new_exercise_id', 'relation', 'new_challenge_id']].values])
# relation_list.append(['exercise-challenge', r_id])

r_id += 1
selected_exe_teacher['new_exercise_id'] = selected_exe_teacher['exercise_id'].map(video_id_dict)
selected_exe_teacher['new_teacher_id'] = selected_exe_teacher['teacher_id'].map(teacher_id_dict)
selected_exe_teacher['relation'] = r_id
kg = np.concatenate([kg, selected_exe_teacher[['new_exercise_id', 'relation', 'new_teacher_id']].values])
relation_list.append(['exercise-teacher', r_id])

# r_id += 1
# selected_stu_ins['new_student_id'] = selected_stu_ins['student_id'].map(user_id_dict)
# selected_stu_ins['new_school_id'] = selected_stu_ins['school_id'].map(school_id_dict)
# selected_stu_ins['relation'] = r_id
# kg = np.concatenate([kg, selected_stu_ins[['new_student_id', 'relation', 'new_school_id']].values])
# relation_list.append(['student-school', r_id])

r_id += 1
selected_sub_dpl_dpl['new_sub_discipline_id'] = selected_sub_dpl_dpl['sub_discipline_id'].map(sub_dpl_id_dict)
selected_sub_dpl_dpl['new_discipline_id'] = selected_sub_dpl_dpl['discipline_id'].map(dpl_id_dict)
selected_sub_dpl_dpl['relation'] = r_id
kg = np.concatenate([kg, selected_sub_dpl_dpl[['new_sub_discipline_id', 'relation', 'new_discipline_id']].values])
relation_list.append(['sub_discipline-discipline', r_id])

r_id += 1
selected_teacher_ins['new_teacher_id'] = selected_teacher_ins['teacher_id'].map(teacher_id_dict)
selected_teacher_ins['new_school_id'] = selected_teacher_ins['school_id'].map(school_id_dict)
selected_teacher_ins['relation'] = r_id
kg = np.concatenate([kg, selected_teacher_ins[['new_teacher_id', 'relation', 'new_school_id']].values])
relation_list.append(['teacher-school', r_id])

In [79]:
selected_user_action_df['new_user_id'] = selected_user_action_df['user_id'].map(user_id_dict)
selected_user_action_df['new_video_id'] = selected_user_action_df['exercise_id'].map(video_id_dict)
selected_user_action_df.sort_values(by='open_time', inplace=True)

selected_user_videos_dict = dict(selected_user_action_df[['new_user_id','new_video_id']]\
                                .groupby('new_user_id')['new_video_id']\
                                .apply(list))
# selected_video_video_ua = []
# for videos in selected_user_videos_dict.values():
#     selected_video_video_ua.extend(np.column_stack((videos[:-2], videos[1:-1])))
# selected_ua_df = pd.DataFrame(selected_video_video_ua, columns=['s_video_id', 'e_video_id'])

# # the orders between user action videos
# r_id += 1
# selected_ua_df['relation'] = r_id
# kg = np.concatenate([kg, selected_ua_df[['s_video_id', 'relation', 'e_video_id']]])
# relation_list.append(['s_video_id-e_video_id-user_action', r_id])

In [80]:
course_exe = pd.read_csv('./knowledgeGraph/relation/exercise_course.csv')

In [81]:
selected_course_exe = course_exe[course_exe['exercise_id'].isin(selected_exe_set)].drop_duplicates("exercise_id", keep='first')

In [82]:
selected_course_exe_g = selected_course_exe.groupby('course_id').agg({'exercise_id': list}).reset_index()

In [83]:
exe_exe_pair_list = []

for _, line in selected_course_exe_g.iterrows():
    exes = line[1]
    exes_encoder = list(map(lambda x : video_id_dict[x], exes))

    selected_exe_start = exes_encoder[:-1]
    selected_exe_end = exes_encoder[1:]
    selected_exe_exe_map = np.column_stack((selected_exe_start, selected_exe_end))
    
    exe_exe_pair_list.extend(selected_exe_exe_map)

In [84]:
exe_sub_dpl_dict = selected_sub_dpl_exe.set_index(['new_exercise_id'])['new_sub_discipline_id'].to_dict()

In [85]:
len(set(video_id_dict.values()) - set(exe_sub_dpl_dict.keys()))

144

In [86]:
if is_save:
    data2txt(kg, '../kg_final.txt')
    data2txt(relation_list, '../relation_list.txt')
    data2txt(exe_exe_pair_list, '../video_video_list.txt')

    with open('../video_course_dict.pkl', 'wb') as f:
        pkl.dump(exe_sub_dpl_dict, f)

In [87]:
train_data = []
test_data = []

for user, videos in tqdm(selected_user_videos_dict.items()):
    # train_len = int(len(videos) * 0.8)
    # test_len = len(videos) - train_len
    # train_candidates = random.sample(videos, train_len)
    # test_candidates = list(filter(lambda x: x not in train_candidates, videos))
    train_data.append([user] + videos[:-1])
    test_data.append([user, videos[-1]])

100%|██████████| 44523/44523 [00:00<00:00, 850114.70it/s]


In [88]:
if is_save:
    data2txt(train_data, '../train.txt')
    data2txt(test_data, '../test.txt')

In [ ]:
RecBole Dataset

In [89]:
rec_data = selected_user_action_df[['new_user_id', 'new_video_id', 'open_time']]

In [90]:
rec_data.columns = ["user_id:token", "item_id:token", 'timestamp:float']

In [91]:
rec_data['timestamp:float'] = pd.to_datetime(rec_data['timestamp:float'])

In [93]:
rec_data['timestamp:float'] = rec_data['timestamp:float'].astype('int64') // 1e9
rec_data['timestamp:float'] = rec_data['timestamp:float'].astype('int64')

In [94]:
rec_data

,user_id:token,item_id:token,timestamp:float
117245,1538,120,1514766284
228493,1462,231,1514766933
272722,1409,37,1514793243
125525,1585,62,1514795184
5642,1525,20,1514795378
...,...,...,...
510915,31419,137,1577750150
92170,32900,32,1577750174
2322068,32507,166,1577750253
1882419,8372,994,1577750279


In [96]:
rec_data.to_csv('./recbole/mooper.inter', sep='\t', index=False)